<a href="https://colab.research.google.com/github/LaisHott/credit-default-analysis/blob/main/Notebook_Projeto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise do risco de inadimplência dos mutuários

Seu projeto é preparar um relatório para a divisão de empréstimos de um banco. Você precisará descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

Seu relatório será considerado ao construir a **pontuação de crédito** de um cliente em potencial. A **pontuação de crédito** é usada para avaliar a capacidade de um devedor em potencial de pagar seu empréstimo.

- O propósito do projeto é desenvolver uma pontuação de crédito para avaliar a capacidade de um devedor em potencial de pagar um empréstimo. Com base nisso, as seguintes hipóteses serão testadas:
    - Hipótese 1 - Estado Civil e Inadimplência: Acredita-se que o estado civil de um cliente possa influenciar sua probabilidade de inadimplência em um empréstimo. Será investigado se existem diferenças significativas na taxa de inadimplência entre clientes solteiros, casados, divorciados e viúvos.
    - Hipótese 2 - Número de Filhos e Inadimplência: Suspeita-se que o número de filhos de um cliente possa afetar sua capacidade de pagar em dia um empréstimo. Será analisado se há uma relação entre o número de filhos e a taxa de inadimplência, explorando se clientes com mais filhos têm uma probabilidade maior de inadimplência.
- Essas hipóteses serão testadas usando análise de dados, como agrupamentos, cálculos de taxa de inadimplência e comparações entre grupos. Com base nos resultados, será possível desenvolver um modelo preditivo que leve em consideração o estado civil e o número de filhos de um cliente para prever a probabilidade de inadimplência. O relatório final será fornecido à divisão de empréstimos do banco.

## Abra o arquivo de dados e veja a informação geral.

In [ ]:
# Carregando todas as bibliotecas
import pandas as pd
import numpy as np

In [ ]:
# Carregue os dados
dados = pd.read_csv('credit_scoring_eng.csv')
dados

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


## Tarefa 1. Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo

In [ ]:
# Vamos ver quantas linhas e colunas nosso conjunto de dados tem
dados.shape

(21525, 12)

In [ ]:
# vamos imprimir as primeiras N linhas
dados.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


- Ao analisar a amostra de dados exibida, foram observados alguns pontos que podem exigir investigação e alterações adicionais:
    - Valores negativos na coluna "days_employed": Foi identificado que existem valores negativos nessa coluna, que representam a quantidade de dias de trabalho. Essa situação é incomum, pois não deveria haver dias de trabalho negativos. Será necessário investigar a origem desses valores e corrigir possíveis erros de registro.
    - Dados ausentes: Foi identificada a presença de dados ausentes em algumas colunas. Essa falta de informações pode afetar a precisão das análises e modelagem futura. Será importante investigar a razão pela qual esses dados estão faltando e decidir qual abordagem tomar para lidar com eles, como imputação de valores ou exclusão das linhas correspondentes.
    - Nomes com grafias diferentes: Foi identificado que existem nomes semelhantes, mas com grafias diferentes. Isso pode dificultar a análise correta dos dados, especialmente quando se trata de agrupamento ou comparação de registros. Será necessário fazer uma verificação cuidadosa e padronizar os nomes para garantir a consistência dos dados.

- Temos 3 situações que requereram tratamento mais adiante:
    - Valores de "dob_years" iguais a zero: Observei registros em que a idade do cliente ("dob_years") está registrada como zero. Isso pode indicar dados ausentes ou erros na coleta. Vamos identificar a origem desses valores e realizar imputação necessária com base em métodos como a mediana ou média das idades válidas.
    - Valores muito altos em "days_employed": Foi observado que a coluna "days_employed" contém valores extremamente altos, que parecem ser inconsistentes com a realidade. Esses valores podem ser resultado de erros de digitação ou problemas na forma como os dados foram registrados. Vamos avaliar a origem desses valores e realizar correções ou remoção desses registros.
    - Registros com número de filhos igual a -1 e 20: Foi identificado que alguns registros possuem um número de filhos igual a -1 ou 20. Esses valores podem ser considerados atípicos ou erros de digitação. Vamos investigar adiante a causa dessas inconsistências e decidir se os registros devem ser corrigidos, removidos ou tratados de alguma forma específica.

In [ ]:
# Obter informações sobre dados
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


- Os valores ausentes estão presentes nas colunas "days_employed" e "total_income". Além disso, a coluna "days_employed" está com um tipo de dado float, o que é inconsistente, pois deveria representar o número de dias trabalhados.

In [ ]:
# Vejamos a tabela filtrada com valores ausentes
dados.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
# Vejamos a tabela filtrada com valores ausentes na primeira coluna com dados ausentes
dados.loc[dados['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


- Embora os valores ausentes possam parecer simétricos à primeira vista, não podemos afirmar com certeza que são simétricos sem realizar investigações adicionais. É importante contar os valores ausentes em todas as linhas com valores ausentes para confirmar se as amostras ausentes são do mesmo tamanho e analisar as características dessas amostras ausentes para obter uma compreensão mais completa.

In [ ]:
# Vamos aplicar várias condições para filtrar dados e observar o número de linhas na tabela filtrada.
# Linhas onde days_employed e total_income é nulo
dados.loc[dados['days_employed'].isna() & dados['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [ ]:
#Verificando simetria
filtro1 = dados['total_income'].isna()
dados[~filtro1]['days_employed'].isna().sum()

0

In [ ]:
filtro2 = dados['days_employed'].isna()
dados[~filtro2]['total_income'].isna().sum()

0

In [ ]:
# Verificando em %, quantos dados NaN existem na coluna em relação ao total de linhas
dados['days_employed'].isna().sum()/dados.shape[0]*100

10.099883855981417

**Conclusão intermediária**

- A quantidade de linhas na tabela filtrada corresponde ao número de valores ausentes na coluna "days_employed".
- Os dois filtros retornaram valor zero, isso significa que não há valores ausentes na coluna 'total_income' para os quais a coluna 'days_employed' esteja preenchida, e também não há valores ausentes na coluna 'days_employed' para os quais a coluna 'total_income' esteja preenchida. Isso pode indicar que os valores ausentes em cada coluna são independentes um do outro, e sua presença ou ausência não está relacionada.

- A porcentagem de valores ausentes em relação ao conjunto de dados total é de aproximadamente 10%, o que não é considerado um valor significativamente grande. No entanto, ainda é importante investigar e tratar esses valores ausentes.
- Os valores ausentes podem estar relacionados a características específicas dos clientes, como tipo de emprego ou outras informações relacionadas à renda. É necessário identificar qual característica pode ser o motivo dos dados ausentes.
- Além disso, é importante verificar se existe alguma dependência entre os valores ausentes e outras variáveis nos dados. Neste caso, observamos que os valores ausentes em "days_employed" correspondem aos valores ausentes em "total_income", o que indica uma possível relação entre essas variáveis.
- Essas observações nos ajudarão a tomar decisões sobre como preencher os valores ausentes e tratar as características específicas dos clientes de maneira adequada durante a análise dos dados.

- Os próximos passos envolvem a análise das características específicas dos clientes relacionadas aos valores ausentes, como tipo de emprego e estado civil, e verificar se há dependência entre os valores ausentes e outras variáveis. Isso nos ajudará a entender melhor a natureza dos valores ausentes e a decidir como preenchê-los adequadamente. Essas ações são fundamentais para realizar uma análise mais precisa dos dados e tomar decisões informadas.

In [ ]:
# Vamos investigar clientes que não possuem dados sobre as características identificadas e a coluna com os valores ausentes
missing_values = dados.loc[dados['days_employed'].isna() & dados['total_income'].isna()]
missing_values

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [ ]:
#Verificar a distribuição
missing_values['income_type'].value_counts(dropna=False,normalize=True)*100

employee         50.827967
business         23.367065
retiree          18.997240
civil servant     6.761730
entrepreneur      0.045998
Name: income_type, dtype: float64

In [ ]:
missing_values['days_employed'].value_counts(dropna=False,normalize=True)*100

NaN    100.0
Name: days_employed, dtype: float64

In [ ]:
#Proporcao de valores ausentes em relacao ao total de dados
missing_values.isna().sum()/dados.shape[0]*100

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

- Em relação à coluna 'days_employed', observamos a presença de valores negativos, o que é inconsistente, já que estamos lidando com dias trabalhados.
- Identificamos a presença de nomes com grafias diferentes, o que pode dificultar a análise e a categorização dos dados.

**Possíveis motivos para valores ausentes nos dados**
- Com base na análise realizada até o momento, não há evidências de um padrão claro nos valores ausentes da coluna total_income em relação às características específicas dos clientes. A ausência de diferenças significativas na proporção de valores ausentes entre diferentes categorias de income_type sugere que os valores ausentes podem ocorrer de forma aleatória e não estão fortemente relacionados a nenhuma característica específica. No entanto, é necessário realizar investigações adicionais e considerar outras hipóteses para obter uma compreensão mais completa dos motivos pelos quais esses valores podem estar em falta.

In [ ]:
# Verificar a distribuição em todo o conjunto de dados
dados.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [ ]:
#Verificando distribuição na tabela filtrada
missing_values.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


**Conclusão intermediária**
- Os valores médios e medianos da tabela filtrada são semelhantes aos do conjunto de dados original, o que sugere que a distribuição dos dados ausentes é semelhante à distribuição geral dos dados. Isso significa que os valores ausentes não parecem introduzir um viés significativo na distribuição dos dados.

- É possível que os valores ausentes sejam devidos a erros de registro, onde algumas informações não foram corretamente registradas ou foram perdidas durante o processo de coleta de dados. Também pode haver casos em que os dados não estão disponíveis devido à falta de resposta dos clientes ou à falta de registro dessas informações específicas. Além disso, problemas técnicos durante a coleta de dados podem ter impedido a obtenção de algumas informações, resultando em valores ausentes.
- Para investigar se os valores ausentes seguem algum padrão, podemos analisar se existem correlações entre os valores ausentes e outras variáveis, como idade, educação ou tipo de renda. Isso pode nos ajudar a identificar se há algum motivo específico para a ausência de dados e se os valores ausentes estão relacionados a características particulares dos clientes.

In [ ]:
# Verifique outros motivos e padrões que possam levar a valores ausentes
#Avaliando a caracteristica número de filho
faltantes_por_children = missing_values.groupby('children')['total_income'].count()
faltantes_por_children

children
-1     0
 0     0
 1     0
 2     0
 3     0
 4     0
 5     0
 20    0
Name: total_income, dtype: int64

In [ ]:
#Avaliando o status familiar
faltantes_por_status = missing_values.groupby('family_status')['total_income'].count()
faltantes_por_status

family_status
civil partnership    0
divorced             0
married              0
unmarried            0
widow / widower      0
Name: total_income, dtype: int64

In [ ]:
#Avaliando propósito do empréstimo
faltantes_por_purpose = missing_values.groupby('purpose')['total_income'].count()
faltantes_por_purpose

purpose
building a property                         0
building a real estate                      0
buy commercial real estate                  0
buy real estate                             0
buy residential real estate                 0
buying a second-hand car                    0
buying my own car                           0
buying property for renting out             0
car                                         0
car purchase                                0
cars                                        0
construction of own property                0
education                                   0
getting an education                        0
getting higher education                    0
going to university                         0
having a wedding                            0
housing                                     0
housing renovation                          0
housing transactions                        0
profile education                           0
property                  

**Conclusão intermediária**
- Com base em nossa análise até o momento, não encontramos evidências que sustentem a presença de um padrão significativo nos valores ausentes. Parece que os valores ausentes são acidentais e não estão fortemente relacionados a nenhuma característica específica dos clientes.
- A proporção de valores ausentes é relativamente baixa em relação ao tamanho do conjunto de dados, o que indica que a perda de informações não é substancial.
- Além disso, examinamos a distribuição dos valores para as variáveis com valores ausentes em relação aos valores das mesmas variáveis, mas não encontramos diferenças significativas que possam indicar um motivo específico para os valores ausentes.


In [ ]:
# Verificando outros padrões - explique quais
# Distribuição dos valores ausentes de 'total_income' de acordo com nivel de escolaridade
faltantes_por_education = missing_values.groupby('education')['total_income'].count()
faltantes_por_education


education
BACHELOR'S DEGREE      0
Bachelor's Degree      0
PRIMARY EDUCATION      0
Primary Education      0
SECONDARY EDUCATION    0
SOME COLLEGE           0
Secondary Education    0
Some College           0
bachelor's degree      0
primary education      0
secondary education    0
some college           0
Name: total_income, dtype: int64

In [ ]:
#Distribuição dos valores ausentes de 'total_income' de acordo com gênero
faltantes_por_gender = missing_values.groupby('gender')['total_income'].count()
faltantes_por_gender


gender
F    0
M    0
Name: total_income, dtype: int64

In [ ]:
#Distribuição dos valores ausentes de 'total_income' de acordo com income type
faltantes_por_income = missing_values.groupby('income_type')['total_income'].count()
faltantes_por_income


income_type
business         0
civil servant    0
employee         0
entrepreneur     0
retiree          0
Name: total_income, dtype: int64

In [ ]:
# Distribuição dos valores ausentes de 'total_income' de acordo com idade
faltantes_por_age = missing_values.groupby('dob_years')['total_income'].count()
faltantes_por_age

dob_years
0     0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
51    0
52    0
53    0
54    0
55    0
56    0
57    0
58    0
59    0
60    0
61    0
62    0
63    0
64    0
65    0
66    0
67    0
68    0
69    0
70    0
71    0
72    0
73    0
Name: total_income, dtype: int64

**Conclusões**
- Após uma análise minuciosa, não encontramos nenhum padrão significativo nos valores ausentes. A proporção de valores ausentes em relação ao tamanho do conjunto de dados é relativamente baixa, e a distribuição dos valores nas outras variáveis não apresenta nenhum comportamento consistente. Com base nessas observações, podemos concluir que os valores ausentes são acidentais e não estão relacionados a nenhuma característica específica dos clientes.


- Abordaremos os valores ausentes preenchendo-os com a mediana da respectiva coluna. A escolha da mediana se deve ao fato de ser uma medida robusta que não é afetada por valores extremos. Isso nos permitirá preservar a distribuição geral dos dados e minimizar o impacto dos valores ausentes em nossas análises.


- As próximas etapas para transformar os dados consistem em:
    - Identificar e remover registros duplicados.
    - Verificar e corrigir quaisquer valores incorretos ou inconsistentes.
    - Preencher os valores ausentes utilizando a mediana da respectiva coluna.
    - Converter variáveis categóricas em variáveis ordinais, se necessário, para facilitar a análise e modelagem dos dados.


## Transformação de dados


- Verificando os dados na coluna 'education'

In [ ]:
# Vamos ver todos os valores na coluna de educação para verificar se e quais grafias precisarão ser corrigidas
dados['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [ ]:
# Corrija os registros, se necessário
wrong_values = {
    "bachelor's degree": "Bachelor's Degree",
    "BACHELOR'S DEGREE": "Bachelor's Degree",
    "secondary education": "Secondary Education",
    "SECONDARY EDUCATION": "Secondary Education",
    "GRADUATE DEGREE": "Graduate Degree",
    "graduate degree": "Graduate Degree",
    "SOME COLLEGE": "Some College",
    "some college": "Some College",
    "primary education": "Primary Education",
    "PRIMARY EDUCATION": "Primary Education",
}


for key, value in wrong_values.items():
    dados['education'] = dados['education'].replace(key,value)


In [ ]:
# Verificando todos os valores na coluna para ter certeza de que os corrigimos
dados['education'].unique()

array(["Bachelor's Degree", 'Secondary Education', 'Some College',
       'Primary Education', 'Graduate Degree'], dtype=object)

- Verificando os dados na coluna 'children'

In [ ]:
# Vamos ver a distribuição de valores na coluna `children`
dados['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [ ]:
dados['children'].count()

21525

In [ ]:
#verificando a quantidade de valores menor que zero na coluna children
dados[dados['children'] < 0 ]['children'].count()

47

In [ ]:
#Porcentagem de dados problemáticos
dados[dados['children'] < 0 ]['children'].count()/dados['children'].count()*100

0.2183507549361208

- Há algo estranho na coluna 'children', existem valores negativos. A porcentagem de dados problemáticos é de 0.21% do total de dados.
- Esses problemas podem ter ocorrido devido a erros de registro, digitação incorreta ou problemas com a entrada de dados no sistema.
- Para lidar com esses dados, uma opção é substituir os valores negativos pelo seu correspondente positivo, assumindo que foi um erro na entrada dos dados. No entanto, essa abordagem pode introduzir outras inconsistências nos dados. Outra opção é excluir esses registros considerando que eles representam uma pequena porcentagem do conjunto de dados. Neste caso, optei por excluir os valores negativos, considerando que eles compõem uma porcentagem muito pequena do dataframe.

In [ ]:
dados_new = dados

In [ ]:
# Corrija os dados com base na sua decisão
dados_new['children'] = dados_new['children'].abs()

In [ ]:
dados_new['children'] = dados_new['children'].replace(20, 2)

In [ ]:
# Verificar a coluna `children` novamente para ter certeza de que está tudo corrigido
sorted(dados_new['children'].unique())

[0, 1, 2, 3, 4, 5]

- Verificando os dados na coluna 'days_employed'

In [ ]:
# Encontre dados problemáticos em `days_employed`, se existirem, e calcule a porcentagem
dados['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

In [ ]:
# Verificando a % de dados negativos da coluna days_employed
data_days_negative = dados[dados['days_employed'] <= 0]['days_employed'].count()/dados['days_employed'].count()
data_days_negative*100

82.19730246498888

In [ ]:
#Mean de valores negativos na coluna days_employed
xn = dados[dados['days_employed'] < 0]['days_employed']
-xn.mean()

2353.0159319988766

In [ ]:
#Mean de valores positivos na coluna days_employed
xp = dados[dados['days_employed'] >= 0]['days_employed']
xp.mean()

365004.3099162686

In [ ]:
#Desvio padrão dos valores positivos
xp.std()

21075.016396377556

In [ ]:
#Desvio padrão dos valores negativos
xn.std()

2304.2438507068005

In [ ]:
xn.max(),xn.min()

(-24.14163324048118, -18388.949900568383)

In [ ]:
xp.max(),xp.min()

(401755.40047533, 328728.72060451825)

In [ ]:
#Substituindo os valores impossíveis de days_employed pela mediana
#Calculando a mediana de valores reais (< 36500) da coluna 'days_employed'
median_days_employed = dados_new.loc[dados_new['days_employed'] < 36500, 'days_employed'].median()

In [ ]:
 median_days_employed

-1630.0193809778218

In [ ]:
#Substituindo os valores irreais pela mediada da coluna considerando os valores reais
dados_new.loc[dados_new['days_employed'] > 36500, 'days_employed'] = median_days_employed

- Considerando que a maior idade que temos no dataframe é de 75 anos, isso equilave aproximadamente a 27.394 dias trabalhados. Tratamos os dados com valores irreais substituindo-os pela mediana calculada para valores considerados reais da coluna 'days_employed'.
- Essa abordagem tem como objetivo tratar valores que parecem ser extremos na coluna 'days_employed', substituindo-os por um valor mais representativo da maioria dos dados válidos, que é a mediana. Isso ajuda a evitar distorções e anomalias nos dados e a garantir uma análise mais precisa.

- A coluna 'days_employed' apresenta uma grande discrepância entre os valores negativos e positivos. Mais de 80% dos dados são negativos, enquanto os valores positivos são significativamente maiores em média. Além disso, o desvio padrão dos valores negativos é menor em comparação aos positivos. Os valores negativos também têm um intervalo mínimo e máximo distintos em relação aos positivos.
- Esses padrões levantam preocupações sobre a integridade e consistência dos dados na coluna 'days_employed'. É possível que os valores negativos representem algum tipo de erro ou problema na coleta ou entrada dos dados. Esses valores negativos extremos, como -18388.95, não fazem sentido na interpretação de dias trabalhados.
- Talvez descartar a variavel days_employed ou transformar em valores absolutos para eliminar negativos.


In [ ]:
# Aborde os valores problemáticos, se existirem
dados_new['days_employed'] = dados_new['days_employed'].abs()

In [ ]:
# Verifique o resultado - certifique-se de que está corrigido
dados_new['days_employed']

0        8437.673028
1        4024.803754
2        5623.422610
3        4124.747207
4        1630.019381
            ...     
21520    4529.316663
21521    1630.019381
21522    2113.346888
21523    3112.481705
21524    1984.507589
Name: days_employed, Length: 21525, dtype: float64

- Verificando a idade do cliente pela coluna 'dob_years'

In [ ]:
# Verifique o `dob_years` para valores suspeitos e conte a porcentagem
dados_new['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [ ]:
dados_new[dados_new['dob_years'] == 0].shape[0]

101

In [ ]:
#Porcentagem de valores iguais a zero
(dados_new[dados_new['dob_years'] == 0]['dob_years'].count() / dados_new['dob_years'].count()) * 100

0.4692218350754936

- Considerando que temos uma pequena porcentagem de valores problemáticos na coluna 'dob_years' (idade igual a zero) e já realizamos exclusões anteriores, é razoável optar por substituir esses valores pela mediana. A substituição pela mediana é menos sensível a valores extremos, e ajuda a preservar a distribuição dos dados.

- Substituir os valores problemáticos pela mediana é uma escolha adequada, pois nos permite manter a integridade do conjunto de dados e evitar distorções nos resultados da análise. Além disso, ao substituir os valores problemáticos pela mediana, ainda podemos aproveitar as informações válidas presentes nos outros atributos desses registros.

In [ ]:
# Resolva os problemas na coluna `dob_years`, se existirem
dob_years_median = dados_new.loc[dados_new['dob_years'] != 0,'dob_years'].median()
dob_years_median

43.0

In [ ]:
dados_new['dob_years'] = dados_new['dob_years'].replace(0,dob_years_median)

In [ ]:
# Verifique o resultado - certifique-se de que está corrigido
dados_new['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

- Verificando a coluna 'family_status'

In [ ]:
# Vamos ver os valores da coluna
dados_new['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [ ]:
# Aborde os valores problemáticos em `family_status`, se eles existirem
#Não foi encontrado valores problemáticos nesta coluna

In [ ]:
# Verifique o resultado - certifique-se de que está corrigido

- Verificando a coluna 'gender'

In [ ]:
# Vamos ver os valores na coluna
dados_new['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [ ]:
# Aborde os valores problemáticos, se existirem
dados_new[dados_new['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,Some College,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


In [ ]:
dados_new = dados_new.drop(dados_new[dados_new['gender'] == 'XNA'].index)

In [ ]:
# Verifique o resultado - certifique-se de que está corrigido
dados_new['gender'].unique()

array(['F', 'M'], dtype=object)

- Verificando a coluna 'income_type'

In [ ]:
# Vamos ver os valores na coluna
dados_new['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

In [ ]:
# Aborde os valores problemáticos, se existirem
#Não foi encontrado valores problemáticos nesta coluna

In [ ]:
# Verifique o resultado - certifique-se de que está corrigido

- Verificando se temos dados duplicados e corrigindo-os

In [ ]:
# Verificar duplicados
dados_new.duplicated().sum()

71

In [ ]:
# Corrija os duplicados, se existirem
dados_new = dados_new.drop_duplicates().reset_index(drop=True)

In [ ]:
# Última verificação se temos duplicados
dados_new.duplicated().sum()

0

In [ ]:
#Verifique o tamanho do conjunto de dados que você tem agora após suas primeiras manipulações com ele
#Validar para saber se aumentou ou diminuiu o tamanho do dataframe
dados_new.shape

(21453, 12)

In [ ]:
dados.shape

(21525, 12)

- O novo conjunto de dados (dados_new) apresenta algumas alterações em relação ao conjunto original:
    - Foram removidos os valores em que o número de filhos era negativo.
    - Os valores zero da coluna 'dob_years' foram substituídos pela mediana da coluna, considerando que não é possível ter clientes com zero anos de idade para análise de empréstimo.
    - Foi removido um valor isolado e não identificado ('XNA') na coluna 'gender'.
    - Foram removidos os dados duplicados.
    - A variável 'days_employed' foi transformada em valores absolutos para prosseguir com a análise.

# Trabalhando com valores ausentes

- Para otimizar o trabalho e melhorar a consistência dos dados, será utilizado um dicionário para a coluna "education". Isso ocorre porque existem diferentes grafias para valores semelhantes, o que pode dificultar a análise e a categorização correta.

- Ao utilizar um dicionário, será possível mapear os diferentes valores para uma forma padronizada, facilitando a análise e o agrupamento dos dados. Por exemplo, existem valores como "Bachelor's", "Bachelor", "bachelor's degree", todos se referindo ao mesmo tipo de educação, o dicionário permitirá padronizar todos eles para uma única forma, evitando duplicações e inconsistências.

In [ ]:
# Encontre os dicionários
education_dic = dados[['education_id', 'education']]
education_dic = education_dic.drop_duplicates().reset_index(drop=True)
education_dic

,education_id,education
0,0,Bachelor's Degree
1,1,Secondary Education
2,2,Some College
3,3,Primary Education
4,4,Graduate Degree


In [ ]:
family_dic = dados[['family_status_id','family_status']]
family_dic = family_dic.drop_duplicates().reset_index(drop=True)
family_dic

,family_status_id,family_status
0,0,married
1,1,civil partnership
2,2,widow / widower
3,3,divorced
4,4,unmarried


### Restaurar valores ausentes em `total_income`

- Colunas com valores ausentes: 'days_employed' e 'total_income'.
    - Para tratar os valores ausentes na coluna "total_income" primeiro, criamos uma função para calcular a categoria de idade com base na coluna "dob_years" e adicionar uma nova coluna. Em seguida, usando uma tabela dinâmica, calcular os valores médios de renda com base nos fatores relevantes. Criar uma função para preencher os valores ausentes com a mediana de renda correspondente ao grupo específico de características e sem seguida aplicar essa função nas linhas com valores ausentes e criar uma nova coluna para armazenar os valores preenchidos.
    - Para tratar os valores ausentes na coluna "days_employed", vamos realizar uma análise da distribuição dessa variável com base nos parâmetros identificados. Criar uma função para calcular a mediana da coluna "days_employed" e aplicar essa função para cada categoria da coluna "income_type". Com os valores de mediana obtidos, substituiu os valores ausentes na coluna "days_employed" pelos respectivos valores de mediana, levando em consideração a categoria de "income_type" de cada registro.

In [ ]:
# Vamos escrever uma função que calcule a categoria de idad
def categorize_age(row):
    age = row['dob_years']
    if age < 18:
        return '0-17'
    elif age <= 30:
        return '18-30'
    elif age <= 45:
        return '31-45'
    elif age <= 60:
        return '46-60'
    else:
        return '61+'

In [ ]:
# Teste se a função funciona
categorize_age(dados_new.iloc[0])

'31-45'

In [ ]:
# Criar coluna nova com base na função
dados_new['age_category'] = dados_new.apply(categorize_age, axis=1)

In [ ]:
# Verificar como os valores estão na nova coluna
dados_new['age_category'].value_counts()

31-45    8587
46-60    7024
18-30    3716
61+      2126
Name: age_category, dtype: int64

- Ao analisar os fatores que normalmente impactam a renda de uma pessoa, como idade, nível de educação, ocupação e experiência profissional, é importante considerar a distribuição desses fatores e avaliar se existem valores extremos ou atípicos que possam distorcer a média dos dados. Nesse contexto, a mediana é uma medida mais adequada para substituir valores ausentes, pois não é influenciada por outliers e é mais robusta a variações nos dados. Portanto, ao lidar com os valores ausentes na coluna de renda, optamos por usar a mediana como referência, pois ela preserva a distribuição dos dados e reduz possíveis distorções nos resultados.

- Criando uma tabela com apenas os dados não nulos para preencher os valores ausentes.

In [ ]:
# Crie uma tabela sem valores ausentes e imprima algumas de suas linhas para garantir que ela fique boa
#Nao quero as linhas onde days_employed é nulo
no_missing_data = dados_new.loc[~dados_new['days_employed'].isnull()]
no_missing_data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_category        0
dtype: int64

In [ ]:
# Veja os valores médios de renda com base nos seus fatores identificados que tem impacto na renda (colunas)
#Pode usar Pivot Table - values='valor que quero agregar'
dados_new.pivot_table(index='income_type',columns='age_category',values='total_income',aggfunc='mean')

age_category,18-30,31-45,46-60,61+
income_type,,,,
business,29344.038250,33381.403020,33079.265344,32500.258163
civil servant,25170.202708,28090.854937,27241.432148,30226.862630
employee,24214.536616,26383.681869,25889.998404,27746.044084
entrepreneur,79866.103000,NaN,NaN,NaN
paternity / maternity leave,NaN,8612.661000,NaN,NaN
retiree,14888.651857,24433.025134,22305.343421,21377.672540
student,15712.260000,NaN,NaN,NaN
unemployed,NaN,21014.360500,NaN,NaN


In [ ]:
#Verificando se Nan correspondia a valor ausente
dados_new[(dados_new['income_type'] == 'entrepreneur') & (dados_new['age_category'] == '31-45')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category


In [ ]:
#Verificando os fatores de fonte de renda e gênero comparando com status familiar
dados_new.pivot_table(index=['income_type','gender'],columns='family_status',values='total_income',aggfunc='mean')


family_status                       civil partnership      divorced  \
income_type                 gender                                    
business                    F            29111.850129  32019.066557   
                            M            37156.671338  40092.561121   
civil servant               F            24410.351416  27619.634643   
                            M            33297.141135  37065.399667   
employee                    F            24143.062155  25278.054901   
                            M            28419.481650  27480.521574   
entrepreneur                F            79866.103000           NaN   
paternity / maternity leave F                     NaN           NaN   
retiree                     F            21537.383254  21637.408467   
                            M            25483.949703  28281.890286   
student                     M                     NaN           NaN   
unemployed                  F            32435.602000           NaN   
                            M                     NaN           NaN   

family_status                            married     unmarried  \
income_type                 gender                               
business                    F       29356.120319  29439.770244   
                            M       37934.581596  34651.520993   
civil servant               F       24455.442277  26841.646341   
                            M       34869.631157  30666.499055   
employee                    F       23261.892596  24927.162596   
                            M       30005.322433  25988.626269   
entrepreneur                F                NaN           NaN   
paternity / maternity leave F        8612.661000           NaN   
retiree                     F       21291.122230  22237.275924   
                            M       23961.412577  23512.900973   
student                     M                NaN  15712.260000   
unemployed                  F                NaN           NaN   
                            M        9593.119000           NaN   

family_status                       widow / widower  
income_type                 gender                   
business                    F          28813.660088  
                            M          27748.843167  
civil servant               F          24966.055225  
                            M          42925.538000  
employee                    F          24121.057815  
                            M          24041.476182  
entrepreneur                F                   NaN  
paternity / maternity leave F                   NaN  
retiree                     F          21281.813855  
                            M          18557.575429  
student                     M                   NaN  
unemployed                  F                   NaN  
                            M                   NaN

In [ ]:
#Verificando o fator de fonte de renda e educação
dados_new.pivot_table(index='income_type',columns='education',values='total_income',aggfunc='mean')

education,Bachelor's Degree,Graduate Degree,Primary Education,Secondary Education,Some College
income_type,,,,,
business,38780.136881,NaN,26409.124931,28718.435242,31620.043969
civil servant,31571.287664,17822.757000,29449.016667,24648.816597,27596.312587
employee,30650.288996,31089.653667,21954.056075,24426.079549,27951.531586
entrepreneur,79866.103000,NaN,NaN,NaN,NaN
paternity / maternity leave,NaN,NaN,NaN,8612.661000,NaN
retiree,27306.878056,28334.215000,17810.387914,21071.829349,22129.937314
student,15712.260000,NaN,NaN,NaN,NaN
unemployed,32435.602000,NaN,NaN,9593.119000,NaN


In [ ]:
#verificando os fatores fonte de renda e total de renda
dados_new.pivot_table(index='income_type', values='total_income', columns='children', aggfunc='mean')

children,0,1,2,3,4,5
income_type,,,,,,
business,32401.126462,32190.504003,32610.332073,33816.952569,23488.48150,29816.2255
civil servant,27919.090476,26225.463459,26365.038181,29140.416917,31861.00100,20176.3440
employee,25828.737657,25765.750817,25591.072475,27695.360922,26996.70463,27668.3966
entrepreneur,79866.103000,NaN,NaN,NaN,NaN,NaN
paternity / maternity leave,NaN,NaN,8612.661000,NaN,NaN,NaN
retiree,21799.966708,23149.672459,26391.040250,25301.815667,24522.21600,NaN
student,15712.260000,NaN,NaN,NaN,NaN,NaN
unemployed,32435.602000,9593.119000,NaN,NaN,NaN,NaN


In [ ]:
# Veja os valores medianos de renda com base nos seus fatores identificados
dados_new.pivot_table(index='income_type',columns='age_category',values='total_income',aggfunc='median')

age_category,18-30,31-45,46-60,61+
income_type,,,,
business,25694.1585,28506.0405,28352.4130,29171.9890
civil servant,23202.8700,24662.7440,23847.2850,24623.8775
employee,21718.0220,23215.3110,22745.1130,23550.5120
entrepreneur,79866.1030,NaN,NaN,NaN
paternity / maternity leave,NaN,8612.6610,NaN,NaN
retiree,12807.0710,20270.0210,19530.7585,18412.9250
student,15712.2600,NaN,NaN,NaN
unemployed,NaN,21014.3605,NaN,NaN


In [ ]:
#Verificando os fatores de tipo de renda e status familiar
dados_new.pivot_table(index='income_type',columns='family_status',values='total_income',aggfunc='median')

family_status,civil partnership,divorced,married,unmarried,widow / widower
income_type,,,,,
business,26819.0775,29206.0190,27956.157,27180.534,23654.9575
civil servant,23629.1115,24360.3255,23718.422,25062.094,21181.7190
employee,22772.5565,22899.7645,23000.976,22095.429,22216.0035
entrepreneur,79866.1030,NaN,NaN,NaN,NaN
paternity / maternity leave,NaN,NaN,8612.661,NaN,NaN
retiree,18818.0015,19262.2350,19077.967,19046.115,18781.2640
student,NaN,NaN,NaN,15712.260,NaN
unemployed,32435.6020,NaN,9593.119,NaN,NaN


In [ ]:
#verificando os fatores de tipo de renda e educação
dados_new.pivot_table(index='income_type',columns='education',values='total_income',aggfunc='median')

education,Bachelor's Degree,Graduate Degree,Primary Education,Secondary Education,Some College
income_type,,,,,
business,32285.6640,NaN,21887.825,25451.3100,28688.018
civil servant,27601.7775,17822.757,23734.287,21864.4750,25694.775
employee,26502.5190,31771.321,20159.186,21848.8175,24209.430
entrepreneur,79866.1030,NaN,NaN,NaN,NaN
paternity / maternity leave,NaN,NaN,NaN,8612.6610,NaN
retiree,23078.5230,28334.215,16415.785,18374.8570,19221.903
student,15712.2600,NaN,NaN,NaN,NaN
unemployed,32435.6020,NaN,NaN,9593.1190,NaN


In [ ]:
#verificando os fatores de tipo de renda e gênero
dados_pivot = dados_new.pivot_table(index='income_type',
                                    columns='gender',
                                    values='total_income',
                                    aggfunc='median')
dados_pivot

gender,F,M
income_type,,
business,25731.3245,31491.0080
civil servant,21917.1980,29754.3915
employee,20898.4980,25945.7880
entrepreneur,79866.1030,NaN
paternity / maternity leave,8612.6610,NaN
retiree,18529.2465,20918.3620
student,NaN,15712.2600
unemployed,32435.6020,9593.1190


In [ ]:
#Outra forma de fazer com formato de tabela diferente
dados_new.groupby('education')['total_income'].mean()

education
Bachelor's Degree      33142.802434
Graduate Degree        27960.024667
Primary Education      21144.882211
Secondary Education    24594.503037
Some College           29040.132990
Name: total_income, dtype: float64

- Ao analisar os fatores que podem influenciar a renda, consideramos diferentes aspectos presentes nas colunas do conjunto de dados, como idade, nível de educação, tipo de emprego e estado civil. Essa análise é importante porque nos permite identificar subgrupos de clientes com características semelhantes e calcular médias ou medianas específicas para cada subgrupo. Essa abordagem nos ajuda a preencher os valores ausentes de forma mais precisa, levando em consideração as diferenças nos fatores que afetam a renda. Por exemplo, é razoável supor que a idade tenha um impacto significativo na renda, assim como o nível de educação e o tipo de emprego. Portanto, ao realizar as comparações, levamos em conta essas variáveis-chave e seu potencial efeito na determinação da renda dos clientes.

- Ao analisar as tabelas, é difícil determinar com precisão qual característica individualmente define mais a renda. No entanto, considerando as informações disponíveis, podemos inferir que uma combinação de diferentes fatores é mais relevante para determinar a renda de uma pessoa. Características como o número de filhos (children), estado civil (family_status) e tipo de emprego (income_type) podem desempenhar um papel significativo na determinação da renda.
- Quanto à escolha entre usar a mediana ou a média para substituir os valores ausentes, devemos levar em consideração a distribuição dos dados e a presença de possíveis outliers. Se a distribuição for aproximadamente simétrica e não houver valores extremos que distorçam significativamente os dados, a média pode ser uma opção adequada. No entanto, se houver uma distribuição assimétrica ou a presença de valores extremos que possam distorcer os resultados, a mediana é uma escolha melhor, pois ela é menos afetada por valores extremos e preserva melhor a tendência central dos dados.

In [ ]:
#Escreva uma função que usaremos para preencher os valores ausentes
#Usando a tabela pivot para preencher os valores ausentes o resultado e a mediana para esse grupo de pessoas com essas caracteristicas
def aux_fill_nan(gender, income_type):
    try:
        return dados_pivot[gender][income_type]
    except:
        return 'Erro'


In [ ]:
#Acessando um valor específico na tabela pivot criada
dados_pivot['F']['business']

25731.324500000002

In [ ]:
dados_pivot['M']

income_type
business                       31491.0080
civil servant                  29754.3915
employee                       25945.7880
entrepreneur                          NaN
paternity / maternity leave           NaN
retiree                        20918.3620
student                        15712.2600
unemployed                      9593.1190
Name: M, dtype: float64

In [ ]:
# Verifique se funciona
aux_fill_nan(gender='F',income_type='business')


25731.324500000002

In [ ]:
# Aplique em todas as linhas
dados_new['mediana_total_income'] = dados_new.apply(lambda row: aux_fill_nan(income_type=row['income_type'],
                                                                             gender=row['gender']),
                                                    axis=1)


In [ ]:
#Substituindo os valores
dados_new['total_income'] = dados_new['total_income'].fillna(dados_new['mediana_total_income'])

In [ ]:
# Verifique se temos algum erro
dados_new[dados_new['total_income'] == 'Erro']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,mediana_total_income


- Não foi encontrado erro ao preparar os valores para dados ausentes.

In [ ]:
# Substituir valores ausentes se houver algum erro

In [ ]:
#Verificando se o tipo da coluna esta correto
dados_new['total_income'].dtype

dtype('float64')

In [ ]:
dados_new[dados_new['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,mediana_total_income
5931,0,NaN,58,Bachelor's Degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,46-60,NaN


In [ ]:
#Preenchendo os dados Nan da couna total_income com a mediana dos valores de 'M'
dados_new['total_income'] = dados_new['total_income'].fillna(dados_new[dados_new['gender'] == 'M']['total_income'].median())


In [ ]:
dados_new['total_income'].isna().sum()

0

- Verificando se o número total de valores na coluna 'total_income' corresponde ao número de valores em outras colunas

In [ ]:
# Verificar o número de entradas nas colunas
dados_new.count()

children                21453
days_employed           19350
dob_years               21453
education               21453
education_id            21453
family_status           21453
family_status_id        21453
gender                  21453
income_type             21453
debt                    21453
total_income            21453
purpose                 21453
age_category            21453
mediana_total_income    21452
dtype: int64

###  Restaurar valores em `days_employed`

- Para restaurar os valores ausentes na coluna 'days_employed', podemos considerar parâmetros como a idade, o tipo de emprego e o estado civil, que podem influenciar a renda. Em seguida, faremos uma análise da distribuição dos dados para determinar se devemos usar a média ou a mediana na substituição dos valores ausentes.

In [ ]:
# Distribuição de `days_employed` medianos com base nos seus parâmetros identificados
dados_days_pivot = dados_new.pivot_table(index=['income_type','family_status'],
                                         columns='gender',
                                         values='days_employed',
                                         aggfunc='median')
dados_days_pivot


gender                                                   F            M
income_type                 family_status                              
business                    civil partnership  1406.527738  1359.016964
                            divorced           1520.532918  1837.252685
                            married            1686.419677  1587.696641
                            unmarried          1305.150018  1069.370722
                            widow / widower    2757.584181  1549.012430
civil servant               civil partnership  2645.791195  2412.218805
                            divorced           2979.425067  2751.423468
                            married            2758.292954  2914.292293
                            unmarried          2165.130798  2103.355440
                            widow / widower    2830.224607  3046.540954
employee                    civil partnership  1726.948357  1289.359088
                            divorced           1629.346256  1319.492928
                            married            1809.885076  1530.385958
                            unmarried          1294.277533  1020.749261
                            widow / widower    2083.456926  1293.823205
entrepreneur                civil partnership   520.848083          NaN
paternity / maternity leave married            3296.759962          NaN
retiree                     civil partnership  1630.019381  1630.019381
                            divorced           1630.019381  1630.019381
                            married            1630.019381  1630.019381
                            unmarried          1630.019381  1630.019381
                            widow / widower    1630.019381  1630.019381
student                     unmarried                  NaN   578.751554
unemployed                  civil partnership  1630.019381          NaN
                            married                    NaN  1630.019381

In [ ]:
# Distribuição de 'days_employed' médios com base nos seus parâmetros identificados
dados_new.pivot_table(index=['income_type','family_status'],
                                         columns='gender',
                                         values='days_employed',
                                         aggfunc='mean')

gender                                                   F            M
income_type                 family_status                              
business                    civil partnership  2020.299637  1979.626325
                            divorced           2139.071509  2464.725924
                            married            2259.179766  2120.650800
                            unmarried          1904.328947  1499.532900
                            widow / widower    3296.970459  1623.387558
civil servant               civil partnership  3334.153552  2872.768769
                            divorced           3578.869543  3027.030399
                            married            3612.612263  3273.707837
                            unmarried          2853.686787  2455.142357
                            widow / widower    4284.836524  3046.540954
employee                    civil partnership  2456.194355  1849.608254
                            divorced           2556.298561  2002.291519
                            married            2609.963282  2209.984493
                            unmarried          2108.206367  1462.079977
                            widow / widower    2884.586465  1891.164923
entrepreneur                civil partnership   520.848083          NaN
paternity / maternity leave married            3296.759962          NaN
retiree                     civil partnership  1630.019381  1630.019381
                            divorced           1630.019381  1630.019381
                            married            1630.019381  1630.019381
                            unmarried          1630.019381  1630.019381
                            widow / widower    1630.019381  1630.019381
student                     unmarried                  NaN   578.751554
unemployed                  civil partnership  1630.019381          NaN
                            married                    NaN  1630.019381

- Irei utilizar a mediana, pois ela é uma medida robusta que não é influenciada por valores extremos, garantindo uma estimativa mais estável e representativa dos valores ausentes.

In [ ]:
# Vamos escrever uma função que calcule médias ou medianas (dependendo da sua decisão) com base no seu parâmetro identificado
def calc_median(gender, income_type, family_status):
    try:
        return dados_days_pivot[gender][income_type][family_status]
    except:
        return 'Error'


In [ ]:
# Verifique se a função funciona
calc_median(gender='F',income_type='business', family_status='civil partnership')

1406.527737503944

In [ ]:
# Aplicar função ao income_type
dados_new['days_employed'] = dados_new.apply(lambda row: calc_median(gender=row['gender'],
                                                                     income_type=row['income_type'],
                                                                     family_status=row['family_status']),
                                             axis=1)

In [ ]:
#Filtro para homens na coluna gender e coluna days_employed != da string 'Error'
dados_new[(dados_new['gender'] == 'M') & (dados_new['days_employed'] != 'Error')]['days_employed'].median()

1530.3859575269482

In [ ]:
#substituindo os valores 'Error' na coluna 'days_employed' pela mediana dos valores da coluna 'days_employed'
dados_new['days_employed'] = dados_new['days_employed'].replace('Error',dados_new[(dados_new['gender'] == 'M') & (dados_new['days_employed'] != 'Error')]['days_employed'].median())


In [ ]:
#Verificando error
dados_new[dados_new['days_employed'] == 'Error']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,mediana_total_income


In [ ]:
# Verifique se a função funcionou
dados_new['days_employed'].isnull().sum()

0

In [ ]:
# Substituir valores ausentes
dados_new['days_employed'].isna().sum()

0

<div class="alert alert-block alert-success">
<b>Comentário da Revisora (revisão 1)</b> <a class="tocSkip"></a>
    
Boa.
</div>

- Verificando se o número total de valores na coluna 'days_employed' corresponde ao número de valores em outras colunas.

In [ ]:
# Verifique as entradas em todas as colunas - certifique-se de corrigir todos os valores ausentes
dados_new.count()

children                21453
days_employed           21453
dob_years               21453
education               21453
education_id            21453
family_status           21453
family_status_id        21453
gender                  21453
income_type             21453
debt                    21453
total_income            21453
purpose                 21453
age_category            21453
mediana_total_income    21452
dtype: int64

In [ ]:
dados_new.isna().sum()

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
age_category            0
mediana_total_income    1
dtype: int64

- O único valor discrepante na coluna "mediana_total_income"apresentando um valor a menos é devido a um caso específico que não existe no conjunto de dados(já verificado quando a coluna foi criada). Essa diferença não está relacionada à ausência de valores, mas sim a um caso isolado que não foi considerado na criação dessa coluna. É importante mencionar que essa discrepância não afeta a análise geral dos dados e que a coluna "mediana_total_income" ainda fornece informações relevantes sobre a mediana da renda total dos clientes.

In [ ]:
print(dados_new.dtypes)

children                  int64
days_employed           float64
dob_years                 int64
education                object
education_id              int64
family_status            object
family_status_id          int64
gender                   object
income_type              object
debt                      int64
total_income            float64
purpose                  object
age_category             object
mediana_total_income    float64
dtype: object


#### Substituindo os dados do tipo número real para tipo inteiro

In [ ]:
dados_new['days_employed'] = dados_new['days_employed'].astype(int)
dados_new['dob_years'] = dados_new['dob_years'].astype(int)
dados_new['total_income'] = dados_new['total_income'].astype(int)
dados_new['mediana_total_income'] = dados_new['mediana_total_income'].fillna(0).astype(int)

In [ ]:
print(dados_new.dtypes)

children                 int64
days_employed            int64
dob_years                int64
education               object
education_id             int64
family_status           object
family_status_id         int64
gender                  object
income_type             object
debt                     int64
total_income             int64
purpose                 object
age_category            object
mediana_total_income     int64
dtype: object


## Categorização de dados

- Para responder às perguntas e testar as hipóteses, categorizar os dados pode ser útil para facilitar a análise e a compreensão dos padrões e relações presentes nos dados. A categorização permite agrupar os dados em diferentes categorias ou grupos, o que pode ajudar a identificar tendências e realizar comparações entre os grupos.

In [ ]:
dados_new.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,mediana_total_income
0,1,1809,42,Bachelor's Degree,0,married,0,F,employee,0,40620,purchase of the house,31-45,20898
1,1,1809,36,Secondary Education,1,married,0,F,employee,0,17932,car purchase,31-45,20898
2,0,1530,33,Secondary Education,1,married,0,M,employee,0,23341,purchase of the house,31-45,25945
3,3,1530,32,Secondary Education,1,married,0,M,employee,0,42820,supplementary education,31-45,25945
4,0,1630,53,Secondary Education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,46-60,18529


In [ ]:
# Imprima os valores dos dados selecionados para categorização
#Categorizando por status familiar
dados_new.groupby('family_status')['total_income'].count()

family_status
civil partnership     4150
divorced              1195
married              12339
unmarried             2810
widow / widower        959
Name: total_income, dtype: int64

In [ ]:
max_family_status = dados_new.groupby('family_status')['total_income'].count().idxmax()
max_family_status

'married'

In [ ]:
#Categorizando pelo número de filhos
dados_new.groupby('children')['total_income'].count()

children
0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: total_income, dtype: int64

In [ ]:
max_children = dados_new.groupby('children')['total_income'].count().idxmax()
max_children

0

In [ ]:
#Categorizando pelo propósito de empréstimo
dados_new.groupby('purpose')['total_income'].count()

purpose
building a property                         619
building a real estate                      624
buy commercial real estate                  661
buy real estate                             620
buy residential real estate                 606
buying a second-hand car                    478
buying my own car                           505
buying property for renting out             651
car                                         494
car purchase                                461
cars                                        478
construction of own property                635
education                                   447
getting an education                        442
getting higher education                    426
going to university                         496
having a wedding                            768
housing                                     646
housing renovation                          607
housing transactions                        652
profile education               

In [ ]:
max_purpose = dados_new.groupby('purpose')['total_income'].count().idxmax()
max_purpose

'wedding ceremony'

- Verificando os valores exclusivos

In [ ]:
# Verifique os valores exclusivos
dados_new['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [ ]:
dados_new['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [ ]:
sorted(dados_new['purpose'].unique())

['building a property',
 'building a real estate',
 'buy commercial real estate',
 'buy real estate',
 'buy residential real estate',
 'buying a second-hand car',
 'buying my own car',
 'buying property for renting out',
 'car',
 'car purchase',
 'cars',
 'construction of own property',
 'education',
 'getting an education',
 'getting higher education',
 'going to university',
 'having a wedding',
 'housing',
 'housing renovation',
 'housing transactions',
 'profile education',
 'property',
 'purchase of a car',
 'purchase of my own house',
 'purchase of the house',
 'purchase of the house for my family',
 'real estate transactions',
 'second-hand car purchase',
 'supplementary education',
 'to become educated',
 'to buy a car',
 'to get a supplementary education',
 'to have a wedding',
 'to own a car',
 'transactions with commercial real estate',
 'transactions with my real estate',
 'university education',
 'wedding ceremony']

- Os principais grupos que aparecem são: married(family_status), sem filhos(children), cerimônia de casamento (purpose).


- Vamos categorizar os dados com base nesses temas. Escolhi a coluna "purpose" para criar a função, pois percebi que existem propósitos semelhantes dispersos nessa coluna. Ao categorizar esses propósitos similares, podemos simplificar a análise e obter uma visão mais clara das principais categorias de propósitos para os quais os clientes solicitam empréstimos. Isso nos permitirá identificar padrões e tendências relevantes para nossa análise.

In [ ]:
# Vamos escrever uma função para categorizar os dados com base em tópicos comuns
#Categorizando pelo propósito
def categorize_purpose(purpose):
    if 'car' in purpose:
        return 'Carro'
    elif 'real estate' in purpose or 'property' in purpose or 'housing' in purpose:
        return 'Imóvel'
    elif 'education' in purpose or 'university' in purpose:
        return 'Educação'
    elif 'wedding' in purpose:
        return 'Casamento'
    else:
        return 'Outros'

In [ ]:
# Crie uma coluna com as categorias e conte os valores para elas
dados['categoria_purpose'] = dados_new['purpose'].apply(categorize_purpose)

In [ ]:
dados['categoria_purpose'].value_counts()

Imóvel       8906
Carro        4306
Educação     3605
Casamento    2324
Outros       2312
Name: categoria_purpose, dtype: int64

<div class="alert alert-block alert-success">
<b>Comentário da Revisora (revisão 1)</b> <a class="tocSkip"></a>
    
Boa.
</div>

In [ ]:
# Examinar todos os dados numéricos em sua coluna selecionada para categorização
dados_new['purpose'].describe()

count                21453
unique                  38
top       wedding ceremony
freq                   791
Name: purpose, dtype: object

In [ ]:
# Obter estatísticas resumidas para a coluna
dados_new['purpose'].describe()

count                21453
unique                  38
top       wedding ceremony
freq                   791
Name: purpose, dtype: object

- Estou usando intervalos de classes de renda para categorizar a coluna "total_income". Defini os limites com base nos quantis da distribuição dos dados:
    - O limiar para a categoria "High" foi definido como a renda mínima dos 5% mais ricos.
    - A categoria "Medium" foi definida com base nos 40% mais ricos dos dados, utilizando o quantil 0.6 da coluna total_income.
- Essa abordagem permite classificar os clientes em diferentes níveis de renda com base em sua posição relativa na distribuição de renda da amostra.

In [ ]:
#Criando variáveis para as faixas de classes de renda
limiar_income_high = np.quantile(dados_new['total_income'], 0.95)
limiar_income_medium_low = np.quantile(dados_new['total_income'], 0.6)
limiar_income_medium_high = np.quantile(dados_new['total_income'],0.2)

In [ ]:
# Criar função para categorização em diferentes grupos numéricos com base em intervalos
#Pensa no problema
def categorize_client(total_income):
    if total_income <= limiar_income_medium_low:
        return 'Low Income'
    elif total_income <= limiar_income_medium_high:
        return 'Below Median Income'
    elif total_income <= limiar_income_high:
        return 'Above Median Income'
    else:
        return 'High Income'


In [ ]:
#Testando a função
categorize_client(17932.802)

'Low Income'

In [ ]:
# Criar coluna com categorias
dados_new['client_category'] = dados_new.apply(lambda row: categorize_client(row['total_income']), axis=1)

In [ ]:
dados_new.groupby('client_category').agg({'debt': ['mean','count']})

debt       
                         mean  count
client_category                     
Above Median Income  0.079316   7136
High Income          0.069897   1073
Low Income           0.083056  13244

In [ ]:
# Conte os valores de cada categoria para ver a distribuição
#Valores calculados na tabela acima com método count()

## Verificar as Hipóteses


**Existe uma correlação entre o nível de renda e do pagamento em dia?**
- Com base nas análises abaixo e nos dados apresentados, podemos observar uma correlação entre o nível de renda e o pagamento em dia. As categorias "High Income" e "Above Median Income" apresentam uma média de pagamento em dia ('debt') menor em comparação com a categoria "Low Income". Isso indica que clientes com renda mais alta tendem a ter uma taxa de inadimplência menor, enquanto clientes com renda mais baixa têm uma taxa de inadimplência maior. Essa correlação sugere que a renda tem uma influência na capacidade dos clientes de cumprir seus compromissos financeiros. No entanto, é importante considerar que outros fatores podem estar envolvidos nessa relação e que essa análise é baseada nos dados fornecidos.

In [ ]:
#Função para analisar a hipótese(coluna em questão) e taxa de endividamento(coluna 'debt')
def cria_tabela(hipotese):
    return dados_new.groupby(hipotese).agg({'debt': ['mean','count']})

In [ ]:
#Verificando os dados entre nível de renda(usando a categorias de income do cliente) e do pagamento em dia
tabela_income_debt = cria_tabela('client_category')
tabela_income_debt*100

debt         
                         mean    count
client_category                       
Above Median Income  7.931614   713600
High Income          6.989748   107300
Low Income           8.305648  1324400

In [ ]:
# Verifique os dados das children e do pagamento em dia(debt)
tabela_children_debt = cria_tabela('children')
tabela_children_debt

debt       
              mean  count
children                 
0         0.075444  14090
1         0.091658   4855
2         0.094925   2128
3         0.081818    330
4         0.097561     41
5         0.000000      9

In [ ]:
# Calcular a taxa de inadimplência com base no número de filhos
#A taxa de inadimplência corresponde a coluna mean da tabela acima
tabela_children_debt['debt']['mean']*100

children
0    7.544358
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: mean, dtype: float64

**Conclusão**

- Com base nas análises realizadas nos dados, concluímos que existe uma relação entre o número de filhos e a taxa de inadimplência. Foi observado que, em geral, a taxa de inadimplência aumenta à medida que o número de filhos aumenta. No entanto, houve uma exceção interessante, onde a taxa de inadimplência foi menor para clientes com 3 filhos em comparação com aqueles com 1 ou 2 filhos. Essa diferença pode estar relacionada a fatores socioeconômicos ou comportamentais específicos desses clientes.


- Além disso, é importante destacar que não foram identificados casos de inadimplência entre os clientes com 5 filhos. Isso pode indicar que esses clientes possuem uma gestão financeira mais cuidadosa ou estão em uma situação financeira mais estável que lhes permite cumprir seus compromissos.

**Existe uma correlação entre o status familiar e o pagamento em dia?**
- Com base nos dados fornecidos e as análises feitas, podemos analisar a possível correlação entre o status familiar e a taxa de inadimplência. Temos clientes solteiros e em parceria civil apresentando uma maior propensão à inadimplência em comparação aos clientes casados e viúvos.


In [ ]:
# Verifique os dados de status da família e do pagamento em dia
tabela_family_status_debt = cria_tabela('family_status')
tabela_family_status_debt

debt       
                       mean  count
family_status                     
civil partnership  0.093494   4150
divorced           0.071130   1195
married            0.075452  12339
unmarried          0.097509   2810
widow / widower    0.065693    959

In [ ]:
# Calcular a taxa padrão com base no status da família
#A taxa corresponde a coluna mean da tabela acima
tabela_family_status_debt['debt']['mean']*100

family_status
civil partnership    9.349398
divorced             7.112971
married              7.545182
unmarried            9.750890
widow / widower      6.569343
Name: mean, dtype: float64

**Conclusão**

- Esses resultados sugerem que o estado civil pode desempenhar um papel na capacidade dos clientes de cumprir seus compromissos financeiros. No entanto, é importante ressaltar que essa correlação não é necessariamente causal e que outros fatores individuais e financeiros podem influenciar a taxa de inadimplência.

- Portanto, é recomendado que instituições financeiras considerem o estado civil dos clientes como um dos fatores ao avaliar o risco de crédito, juntamente com outras informações relevantes. No entanto, é fundamental realizar análises mais aprofundadas e considerar múltiplos fatores antes de tomar decisões definitivas.

**Como a finalidade do crédito afeta a taxa de inadimplência?**
- Com base na análise dos dados, pode-se observar que a finalidade do crédito tem influência na taxa de inadimplência dos clientes.
    - Os resultados mostraram que as finalidades relacionadas à educação, como "getting higher education", "profile education" e "supplementary education", apresentaram taxas médias de inadimplência mais elevadas, variando de 8.30% a 11.46%. Isso pode ser atribuído a fatores como a duração prolongada dos estudos e potenciais dificuldades financeiras enfrentadas pelos indivíduos durante esse período.

- Dessa forma, é recomendado que as instituições financeiras adotem uma abordagem cautelosa ao avaliar solicitações de empréstimo, levando em consideração não apenas a finalidade do crédito, mas também a situação financeira geral do cliente e seu histórico de crédito. Essa análise mais abrangente permitirá uma tomada de decisão mais precisa e uma redução dos riscos associados à inadimplência.

In [ ]:
# Confira os percentuais de inadimplência para cada finalidade de crédito e analise-os
tabela_purpose_debt = cria_tabela('purpose')
tabela_purpose_debt

debt      
                                              mean count
purpose                                                 
building a property                       0.087237   619
building a real estate                    0.076923   624
buy commercial real estate                0.071104   661
buy real estate                           0.069355   620
buy residential real estate               0.067657   606
buying a second-hand car                  0.075314   478
buying my own car                         0.091089   505
buying property for renting out           0.079877   651
car                                       0.085020   494
car purchase                              0.091106   461
cars                                      0.092050   478
construction of own property              0.066142   635
education                                 0.071588   447
getting an education                      0.083710   442
getting higher education                  0.107981   426
going to university                       0.086694   496
having a wedding                          0.083333   768
housing                                   0.071207   646
housing renovation                        0.057661   607
housing transactions                      0.073620   652
profile education                         0.100917   436
property                                  0.066351   633
purchase of a car                         0.109890   455
purchase of my own house                  0.054839   620
purchase of the house                     0.074303   646
purchase of the house for my family       0.070533   638
real estate transactions                  0.081481   675
second-hand car purchase                  0.104938   486
supplementary education                   0.082609   460
to become educated                        0.095588   408
to buy a car                              0.093418   471
to get a supplementary education          0.114350   446
to have a wedding                         0.075817   765
to own a car                              0.100418   478
transactions with commercial real estate  0.080000   650
transactions with my real estate          0.079745   627
university education                      0.088496   452
wedding ceremony                          0.080910   791

In [ ]:
tabela_purpose_debt['debt']['mean']*100

purpose
building a property                          8.723748
building a real estate                       7.692308
buy commercial real estate                   7.110439
buy real estate                              6.935484
buy residential real estate                  6.765677
buying a second-hand car                     7.531381
buying my own car                            9.108911
buying property for renting out              7.987711
car                                          8.502024
car purchase                                 9.110629
cars                                         9.205021
construction of own property                 6.614173
education                                    7.158837
getting an education                         8.371041
getting higher education                    10.798122
going to university                          8.669355
having a wedding                             8.333333
housing                                      7.120743
housing renovation  

**Conclusão**

- Com base nas manipulações e observações realizadas, podemos concluir que a finalidade do crédito pode ter uma influência na taxa de inadimplência, mas não é o único fator determinante. Houve variações na taxa de inadimplência entre diferentes categorias de finalidade, indicando que alguns propósitos podem estar associados a um maior risco de inadimplência.
- É recomendado que as instituições financeiras adotem políticas de avaliação de crédito sólidas e criteriosas, considerando diversos fatores e realizando análises mais detalhadas para identificar e gerenciar os riscos de inadimplência. Isso garantirá uma maior segurança nas operações de crédito e contribuirá para uma gestão eficiente dos recursos financeiros.

# Conclusão Geral


- Durante a análise deste conjunto de dados, identificamos alguns valores ausentes em diversas colunas. Os campos com valores ausentes foram: 'days_employed', 'total_income'. A ausência desses valores pode ser atribuída a diferentes razões, como problemas técnicos durante a coleta de dados, falhas no registro das informações ou clientes que optaram por não fornecer certas informações pessoais.
- Para lidar com os valores ausentes, adotamos diferentes abordagens. Para as colunas 'days_employed' e 'total_income', preenchemos os valores ausentes com as respectivas medianas, com base nas categorias de gênero. Essa escolha foi feita levando em consideração a natureza das variáveis e a presença de possíveis valores discrepantes.
- Quanto à identificação e exclusão de dados duplicados, utilizamos o método 'drop_duplicates()' do pandas. Duplicatas podem ocorrer devido a erros no processo de entrada de dados ou a repetições na coleta dos dados. A remoção desses dados duplicados foi essencial para garantir a integridade e a precisão dos resultados da análise.
- Ao longo da análise, também realizamos alterações nos tipos de dados das colunas convertendo as colunas para o tipo de dado inteiro.
- Para a categorização dos dados, selecionamos dicionários específicos para algumas colunas, como 'education', 'family_status' e 'purpose'. Essa escolha foi baseada na necessidade de agrupar e analisar os dados de maneira mais significativa e compreensível, fornecendo insights sobre a relação entre essas variáveis e a taxa de inadimplência.
- Essas etapas de pré-processamento foram essenciais para garantir a confiabilidade dos resultados e fornecer informações relevantes para a tomada de decisões no contexto financeiro. No entanto, é importante ressaltar que a análise de dados é um processo contínuo e, em futuras análises, outras questões e ajustes podem surgir, exigindo abordagens específicas para obter resultados ainda mais precisos e confiáveis.

#### Com base na análise realizada, podemos tirar as seguintes conclusões em relação às perguntas feitas:

- Ter filhos parece ter uma correlação negativa com o pagamento pontual de um empréstimo. Os clientes que têm filhos têm uma taxa de inadimplência mais alta em comparação com aqueles que não têm filhos. No entanto, é importante ressaltar que esta correlação não é necessariamente causal e outros fatores podem influenciar a taxa de inadimplência.

- O estado civil também parece ter uma correlação negativa com o pagamento pontual de um empréstimo. Os clientes que são divorciados, viúvos ou solteiros têm uma taxa de inadimplência mais alta em comparação com os casados. No entanto, assim como no caso anterior, é importante ressaltar que esta correlação não é necessariamente causal e outros fatores podem influenciar a taxa de inadimplência.

- Em relação ao nível de renda, os dados sugerem que existe uma correlação negativa entre o nível de renda e a taxa de inadimplência. Quanto menor a renda do cliente, maior a taxa de inadimplência. No entanto, é importante ressaltar que esta correlação não é necessariamente causal e outros fatores podem influenciar a taxa de inadimplência.

- A finalidade do empréstimo também parece ter uma influência significativa na taxa de inadimplência. Os clientes que pegam empréstimos para a compra de carros têm uma taxa de inadimplência mais alta em comparação com aqueles que pegam empréstimos para a compra de imóveis ou para educação. No entanto, é importante ressaltar que esta correlação não é necessariamente causal e outros fatores podem influenciar a taxa de inadimplência.

###### Em resumo, a análise dos dados sugere que há várias variáveis que podem influenciar a taxa de inadimplência, como o estado civil, ter filhos, a finalidade do empréstimo e a renda. No entanto, é importante enfatizar que essas correlações não são necessariamente causais e que a análise de risco de empréstimos deve considerar vários fatores antes de tomar uma decisão final.